# Import Modules

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Lasso # That good L1 reg

# Static Variables

In [2]:
# Paths
base_path = 'Datasets\\'
train_data = os.path.join(base_path, 'train.csv')
test_data = os.path.join(base_path, 'test.csv')

# Load Data
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)

In [3]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
# Drop columns that won't be useful
train_df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


# Exploratory Data Analysis / Cleaning

In [5]:
"""
We want to replace Sex values and Embarked values with numbers.
"""
new_sex = train_df['Sex'].replace({'male':0, 'female':1}) #males are 0, females are 1
train_df['Sex'] = new_sex
new_embarked = train_df['Embarked'].replace({'S':0, 'C':1, 'Q':2}) #Pretty self explanatory
train_df['Embarked'] = new_embarked
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0.0
1,1,1,1,38.0,1,0,71.2833,1.0
2,1,3,1,26.0,0,0,7.9250,0.0
3,1,1,1,35.0,1,0,53.1000,0.0
4,0,3,0,35.0,0,0,8.0500,0.0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,0.0
887,1,1,1,19.0,0,0,30.0000,0.0
888,0,3,1,NaN,1,2,23.4500,0.0
889,1,1,0,26.0,0,0,30.0000,1.0


In [6]:
train_df.describe()
# We will have to clean the Age column (over 100 nulls).  
#Embarked isn't missing enough to cause concern; however, we'll need to fill those in before we can move on.

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,0.362205
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.636157
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [7]:
train_df.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,0.543351,-0.077221,-0.035322,0.081629,0.257307,0.108669
Pclass,-0.338481,1.000000,-0.131900,-0.369226,0.083081,0.018443,-0.549500,0.043835
Sex,0.543351,-0.131900,1.000000,-0.093254,0.114631,0.245489,0.182333,0.118593
Age,-0.077221,-0.369226,-0.093254,1.000000,-0.308247,-0.189119,0.096067,0.012186
SibSp,-0.035322,0.083081,0.114631,-0.308247,1.000000,0.414838,0.159651,-0.060606
Parch,0.081629,0.018443,0.245489,-0.189119,0.414838,1.000000,0.216225,-0.079320
Fare,0.257307,-0.549500,0.182333,0.096067,0.159651,0.216225,1.000000,0.063462
Embarked,0.108669,0.043835,0.118593,0.012186,-0.060606,-0.079320,0.063462,1.000000


In [8]:
# It looks like Age has the most correlation 
#(and, if you look at the two null values, that's the only difference between the two)
train_df[train_df['Embarked'].isnull()]
# Lets find the median age for each Embarked value and fill these with whichever is closest

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,1,1,38.0,0,0,80.0,NaN
829,1,1,1,62.0,0,0,80.0,NaN


In [9]:
train_df.groupby('Embarked').median()['Age']
# Looks like they're both being assigned the value of 1.

Embarked
0.0    28.0
1.0    29.0
2.0    27.0
Name: Age, dtype: float64

In [10]:
train_df['Embarked'].fillna(1, inplace=True)

In [11]:
# We'll use linear regression to find the most likely age
train_df.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,0.543351,-0.077221,-0.035322,0.081629,0.257307,0.111203
Pclass,-0.338481,1.000000,-0.131900,-0.369226,0.083081,0.018443,-0.549500,0.040138
Sex,0.543351,-0.131900,1.000000,-0.093254,0.114631,0.245489,0.182333,0.121266
Age,-0.077221,-0.369226,-0.093254,1.000000,-0.308247,-0.189119,0.096067,0.017651
SibSp,-0.035322,0.083081,0.114631,-0.308247,1.000000,0.414838,0.159651,-0.061591
Parch,0.081629,0.018443,0.245489,-0.189119,0.414838,1.000000,0.216225,-0.080277
Fare,0.257307,-0.549500,0.182333,0.096067,0.159651,0.216225,1.000000,0.065492
Embarked,0.111203,0.040138,0.121266,0.017651,-0.061591,-0.080277,0.065492,1.000000


In [12]:
# Set up two DataFrames: One for null age values (to train) and one without (to predict)
no_age_nan = train_df[train_df['Age'].notnull()]
only_age_nan = train_df[train_df['Age'].isnull()]

# Grab from df without null age values to train the alg
X_array = np.array(no_age_nan[['Pclass','Sex','SibSp','Parch','Fare','Embarked']])
y_array = np.array(no_age_nan['Age'])

In [13]:
# Train the alg
lasso = Lasso(alpha=4) 
lasso.fit(X=X_array, y=y_array)
"""
I chose to use the lasso algorithm because I thought it would be more effective to try to regularize while trying to 
estimate the median.  This won't be perfect, but it will avoid NEGATIVE values as well as absurdly young values.  
"""

"\nI chose to use the lasso algorithm because I thought it would be more effective to try to regularize while trying to \nestimate the median.  This won't be perfect, but it will avoid NEGATIVE values as well as absurdly young values.  \n"

In [14]:
predicted_age = lasso.predict(only_age_nan[['Pclass','Sex','SibSp','Parch','Fare','Embarked']])
predicted_age

array([29.21811425, 29.33564896, 29.18619767, 29.18619767, 29.20312772,
       29.20355731, 32.40484344, 29.19978415, 29.18630636, 29.20355731,
       29.20754785, 29.01415463, 29.19978415, 28.78787437, 29.91792672,
       29.71660871, 29.00757619, 29.20355731, 29.20754785, 29.20075461,
       29.20754785, 29.20754785, 29.20355731, 29.20043112, 29.23800808,
       29.20754785, 29.19978415, 29.19164066, 29.39376804, 29.18846208,
       29.22339875, 27.70957223, 30.42256745, 29.67013518, 28.49969927,
       27.70957223, 29.38870093, 30.29317239, 29.01415463, 29.19978415,
       29.19978415, 27.70957223, 28.81359229, 29.20355731, 28.49969927,
       29.19460834, 28.98709036, 29.01415463, 29.18684464, 31.04883953,
       29.19978415, 29.19978415, 29.80147117, 29.19978415, 28.99922181,
       29.67207611, 29.71660871, 29.78853166, 29.19978415, 28.82852512,
       29.3188276 , 29.20754785, 31.86877202, 27.70957223, 28.82852512,
       32.07175986, 29.20355731, 29.02968204, 29.90498721, 29.18

In [15]:
# Now lets fill it in
train_df.loc[train_df['Age'].isnull(),'Age'] = predicted_age
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.000000,1,0,7.2500,0.0
1,1,1,1,38.000000,1,0,71.2833,1.0
2,1,3,1,26.000000,0,0,7.9250,0.0
3,1,1,1,35.000000,1,0,53.1000,0.0
4,0,3,0,35.000000,0,0,8.0500,0.0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,0.0
887,1,1,1,19.000000,0,0,30.0000,0.0
888,0,3,1,29.219893,1,2,23.4500,0.0
889,1,1,0,26.000000,0,0,30.0000,1.0


In [16]:
train_df.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.630654,0.523008,0.381594,32.204208,0.363636
std,0.486592,0.836071,0.477990,13.008302,1.102743,0.806057,49.693429,0.636159
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,0.000000,3.000000,0.000000,29.199137,0.000000,0.000000,14.454200,0.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000
